# Torrevieja Data Integration

In [ ]:
import os
import json
import pandas as pd

## Weather Station

### Input format

JSON file has the following structure:

```json
{
    "series": {
        "temperatura": {
            "sensor-name": [
                [ "unix-timestamp1", "value1" ],
                [ "unix-timestamp2", "value2" ]
            ],
        },
        "humedad": {
            "sensor-name": [
                [ "unix-timestamp", "value" ]
            ],
        },
        "presion": {
            "sensor-name": [
                [ "unix-timestamp", "value" ]
            ],
        }
    }
}
```

### Output format

The output will be a CSV file with the following columns:

- `timestamp`
- `temperature`
- `humidity`
- `pressure`

In [ ]:
folder_path = "json_data/weather-station"
sensor_name = "5rTM-4sB-iVpGSRJcSJV_La Mata - Parque Natural_38.02519_-0.65845_0"
file_name = sensor_name.replace(" ", "_") + "_2024"
file_path = os.path.join(folder_path, file_name + ".json")

with open(file_path) as f:
    data = json.load(f)

In [ ]:
df_temperature = pd.DataFrame(data['series']['temperatura'][sensor_name], columns=['timestamp', 'temperature'])
df_humidity = pd.DataFrame(data['series']['humedad'][sensor_name], columns=['timestamp', 'humidity'])
df_pressure = pd.DataFrame(data['series']['presion'][sensor_name], columns=['timestamp', 'pressure'])

df_temperature.drop_duplicates(subset='timestamp', keep='first', inplace=True)
df_humidity.drop_duplicates(subset='timestamp', keep='first', inplace=True)
df_pressure.drop_duplicates(subset='timestamp', keep='first', inplace=True)

df_temperature['timestamp'] = pd.to_datetime(df_temperature['timestamp'], unit='ms')
df_humidity['timestamp'] = pd.to_datetime(df_humidity['timestamp'], unit='ms')
df_pressure['timestamp'] = pd.to_datetime(df_pressure['timestamp'], unit='ms')

df = pd.merge(df_temperature, df_humidity, how='inner', on='timestamp')
df = pd.merge(df, df_pressure, how='inner', on='timestamp')

df.set_index('timestamp', inplace=True)

In [ ]:
df.to_csv(file_name + ".csv")